In [1]:
!pip install pandas matplotlib seaborn -q
!pip install ipywidgets -q


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from ipywidgets import interact, widgets
from IPython.display import display, clear_output
import matplotlib.dates as mdates

df = pd.read_csv('patient_data.csv')
df['DateTime'] = pd.to_datetime(df['DateTime'])
df['Date'] = df['DateTime'].dt.date
df['Name'] = df['Name'].str.strip()

name_options = sorted(df['Name'].unique())
date_options_by_name = {name: sorted(df[df['Name'] == name]['Date'].unique()) for name in name_options}
metric_options = ['All', 'SpO2', 'Temperature', 'Pulse']

name_dropdown = widgets.Dropdown(options=name_options, description='Name:')
date_dropdown = widgets.Dropdown(description='Date:')
metric_dropdown = widgets.Dropdown(options=metric_options, description='Metric:')

def update_date_options(*args):
    selected_name = name_dropdown.value
    date_dropdown.options = date_options_by_name.get(selected_name, [])

name_dropdown.observe(update_date_options, names='value')
update_date_options()

def plot_metrics(name, date, metric):
    clear_output(wait=True)
    display(name_dropdown, date_dropdown, metric_dropdown)

    filtered = df[
        (df['Name'] == name) &
        (df['Date'] == date)
    ].sort_values('DateTime')

    if filtered.empty:
        print("No data found for that name and date.")
        return

    plt.figure(figsize=(14, 6))

    if metric == 'All':
        plt.plot(filtered['DateTime'], filtered['SpO2'], marker='o', label='SpO₂ (%)')
        plt.plot(filtered['DateTime'], filtered['Temperature'], marker='s', label='Temperature (°C)')
        plt.plot(filtered['DateTime'], filtered['Pulse'], marker='^', label='Pulse (BPM)')
    else:
        plt.plot(filtered['DateTime'], filtered[metric], marker='o', label=metric)

    plt.title(f"{metric} Readings for {name} on {date}" if metric != 'All' else f"Health Metrics for {name} on {date}")
    plt.xlabel("Time")
    plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
    plt.ylabel("Values")
    plt.legend()
    plt.grid(True)
    plt.tight_layout()
    plt.show()

interact(plot_metrics, name=name_dropdown, date=date_dropdown, metric=metric_dropdown)


interactive(children=(Dropdown(description='Name:', options=('Viprakshi', 'Vipul', 'Yashmitha', 'viprakshi', '…

<function __main__.plot_metrics(name, date, metric)>